In [2]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 50.4838,
	"longitude": 5.8667,
	"hourly": ["temperature_2m", "relative_humidity_2m", "rain"],
	"models": "meteofrance_seamless"
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_rain = hourly.Variables(2).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["rain"] = hourly_rain

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe

Coordinates 50.47999954223633°N 5.869998931884766°E
Elevation 310.0 m asl
Timezone NoneNone
Timezone difference to GMT+0 0 s


,date,temperature_2m,relative_humidity_2m,rain
0,2025-07-27 00:00:00+00:00,17.697001,83.0,0.0
1,2025-07-27 01:00:00+00:00,17.347000,80.0,0.0
2,2025-07-27 02:00:00+00:00,17.197001,80.0,0.0
3,2025-07-27 03:00:00+00:00,16.597000,88.0,0.1
4,2025-07-27 04:00:00+00:00,16.597000,91.0,0.0
...,...,...,...,...
163,2025-08-02 19:00:00+00:00,NaN,NaN,NaN
164,2025-08-02 20:00:00+00:00,NaN,NaN,NaN
165,2025-08-02 21:00:00+00:00,NaN,NaN,NaN
166,2025-08-02 22:00:00+00:00,NaN,NaN,NaN


In [2]:
hourly_dataframe.describe()

,temperature_2m,relative_humidity_2m,rain
count,103.000000,103.000000,103.000000
mean,18.049097,81.019417,0.055340
std,2.709885,14.956788,0.138428
min,13.947000,50.000000,0.000000
25%,15.847000,68.500000,0.000000
50%,17.747000,86.000000,0.000000
75%,20.105499,93.500000,0.000000
max,23.796999,100.000000,0.700000
